## Required packages

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from math import pi
from sklearn.metrics import mean_squared_error, r2_score


In [10]:
%run Optimization_RTK_Functions.ipynb

## Min-Max Normalized Metrices with box Plots and Final weighted Score 

In [17]:
# --------------------------
# 1. Data Import and Combination

file_path = 'Metrices_all_algorithms_Ro_E6_CCW.xlsx'  # Replace with your file path

''' Uncomment the individual algorithm or the enseemble configuration for the analysis'''

#algorithms = [ 'CMA-ES']
#algorithms = [ 'DE']
#algorithms = [ 'GA']
#algorithms = [ 'SA']
#algorithms = [ 'PSO']
#algorithms = ['CMA-ES', 'DE', 'GA', 'SA', 'PSO']
#algorithms = ['DE', 'GA', 'SA', 'CMA-ES']
#algorithms = ['DE',  'SA', 'CMA-ES']
algorithms = ['DE',  'SA']



all_data = []
for algo in algorithms:
    df = pd.read_excel(file_path, sheet_name=algo)
    df['Algorithm'] = algo  # Add an identifier column for the algorithm
    all_data.append(df)

# Combine data from all sheets into a single DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# --------------------------
# 2. Preserve Raw Metrics
# --------------------------
combined_df['Raw_RMSE'] = combined_df['RMSE']
combined_df['Raw_R2'] = combined_df['R2']
combined_df['Raw_PBIAS'] = combined_df['PBIAS']
combined_df['Raw_NSE'] = combined_df['NSE']
combined_df['Raw_Fitness'] = combined_df['Fitness']


# --------------------------
# 3. Normalize Metrics
# --------------------------
# For minimization metrics (RMSE and PBIAS), lower values are better so we invert the scale.
combined_df['Normalized_RMSE'] = (combined_df['Raw_RMSE'].max() - combined_df['Raw_RMSE']) / (combined_df['Raw_RMSE'].max() - combined_df['Raw_RMSE'].min())
combined_df['Normalized_PBIAS'] = (combined_df['Raw_PBIAS'].max() - combined_df['Raw_PBIAS']) / (combined_df['Raw_PBIAS'].max() - combined_df['Raw_PBIAS'].min())

# For maximization metrics (R2 and NSE), standard min–max normalization is used.
combined_df['Normalized_R2'] = (combined_df['Raw_R2'] - combined_df['Raw_R2'].min()) / (combined_df['Raw_R2'].max() - combined_df['Raw_R2'].min())
combined_df['Normalized_NSE'] = (combined_df['Raw_NSE'] - combined_df['Raw_NSE'].min()) / (combined_df['Raw_NSE'].max() - combined_df['Raw_NSE'].min())

# Normalize Fitness (minimization metric) for plotting purposes
combined_df['Normalized_Fitness'] = (combined_df['Raw_Fitness'].max() - combined_df['Raw_Fitness']) / (combined_df['Raw_Fitness'].max() - combined_df['Raw_Fitness'].min())


# --------------------------
#  Calculate Final Score
# --------------------------
# Final score is the average of the four normalized metrics (equal weights of 0.25 each)
combined_df['Final_Score'] = (combined_df['Normalized_RMSE'] +
                              combined_df['Normalized_R2'] +
                              combined_df['Normalized_PBIAS'] +
                              combined_df['Normalized_NSE']) / 4

# --------------------------
# Create Boxplots for Visualization
# --------------------------
# Melt the DataFrame for easier plotting of normalized metrics
melted_df = combined_df.melt(
    id_vars=['Run', 'Algorithm'],
    value_vars=['Normalized_RMSE', 'Normalized_R2', 'Normalized_PBIAS', 'Normalized_NSE', 'Normalized_Fitness'],
    var_name='Metric',
    value_name='Value'
)

# LaTeX strings for y-axis labels
normalization_formulas = {
    'Normalized_RMSE': r'$\frac{RMSE_{max} - RMSE}{RMSE_{max} - RMSE_{min}}$',
    'Normalized_PBIAS': r'$\frac{PBIAS_{max} - PBIAS}{PBIAS_{max} - PBIAS_{min}}$',
    'Normalized_R2': r'$\frac{R^2 - R^2_{min}}{R^2_{max} - R^2_{min}}$',
    'Normalized_NSE': r'$\frac{NSE - NSE_{min}}{NSE_{max} - NSE_{min}}$',
    'Normalized_Fitness': r'$\frac{Fitness_{max} - Fitness}{Fitness_{max} - Fitness_{min}}$'
}

palette = sns.color_palette("Set2", len(algorithms))
metrics_4 = ['Normalized_RMSE', 'Normalized_R2', 'Normalized_PBIAS', 'Normalized_NSE']

# --------------------------
# Create Final DataFrame with Average Values for Each Algorithm
# --------------------------
final_avg_df = combined_df.groupby('Algorithm').agg({
    'Raw_RMSE': 'mean',
    'Raw_R2': 'mean',
    'Raw_PBIAS': 'mean',
    'Raw_NSE': 'mean',
    'Normalized_RMSE': 'mean',
    'Normalized_R2': 'mean',
    'Normalized_PBIAS': 'mean',
    'Normalized_NSE': 'mean',
    'Final_Score': 'mean'
}).reset_index()

# Rename columns for clarity
final_avg_df.columns = ['Algorithm', 'Avg_RMSE', 'Avg_R2', 'Avg_PBIAS', 'Avg_NSE',
                          'Avg_Normalized_RMSE', 'Avg_Normalized_R2', 'Avg_Normalized_PBIAS', 'Avg_Normalized_NSE',
                          'Avg_Final_Score']

# print("Final Average DataFrame for Each Algorithm:")
# print(final_avg_df)

'''
## Mean Based Ranking and weights
'''

# Calculate average Final Score for each algorithm
average_final = combined_df.groupby('Algorithm')['Final_Score'].mean().reset_index()

# --- Method 1: Linear Scaling ---
# # Use average Final Score directly
# linear_weights = average_final['Final_Score'] / np.sum(average_final['Final_Score'])
# average_final['Linear_Weights'] = linear_weights

# --- Method 2: Rank-Based Weights ---
# Rank algorithms so that a higher Final Score gets a better rank (rank 1 is best)
ranks = average_final['Final_Score'].rank(ascending=False, method='min')
rank_weights = (len(average_final) - ranks + 1) / np.sum(len(average_final) - ranks + 1)
average_final['Rank_Weights'] = rank_weights

# # --- Method 3: Softmax Function ---
# def softmax(x):
#     exp_x = np.exp(x - np.max(x))  # For numerical stability
#     return exp_x / np.sum(exp_x)
# softmax_weights = softmax(average_final['Final_Score'])
# average_final['Softmax_Weights'] = softmax_weights

# Assign final ranks based on the average Final Score (higher is better)
average_final['Rank'] = average_final['Final_Score'].rank(ascending=False, method='min')

# Sort the DataFrame by rank
average_final = average_final.sort_values(by='Rank')

print("Ranking and Weights based on Final Score:")
print(average_final[['Algorithm', 'Final_Score', 'Rank', 'Rank_Weights']])



Ranking and Weights based on Final Score:
  Algorithm  Final_Score  Rank  Rank_Weights
0        DE     0.742362   1.0      0.666667
1        SA     0.524473   2.0      0.333333


## Reading RTK parameters value for all iteration of all algorithms

In [19]:
# Load the Excel file
#file_path = 'RTK_Parameters_all_algorithms.xlsx'
file_path = 'RTK_Parameters_all_algorithms_Ro_constraint_E6_CCW.xlsx'

# Dictionary to hold data for each algorithm
algorithm_data = {}

# Read each sheet and process the data
for algorithm in algorithms:
    # Read the sheet into a DataFrame
    df = pd.read_excel(file_path, sheet_name=algorithm)
    
    # List to hold tuples for each run
    runs = []
    
    # Iterate over each row
    for index, row in df.iterrows():
        # Extract (R1, T1, K1), (R2, T2, K2), (R3, T3, K3) tuples
        tuple1 = (row['R1'], row['T1'], row['K1'])
        tuple2 = (row['R2'], row['T2'], row['K2'])
        tuple3 = (row['R3'], row['T3'], row['K3'])
        
        # Append the tuples to the runs list
        runs.append((tuple1, tuple2, tuple3))
    
    # Store the runs in the dictionary
    algorithm_data[algorithm] = runs


## Select Event

In [37]:
#file_path = './Tamucc_event_4.xlsx'
file_path = './CCW_event_6.xlsx'
data=pd.read_excel(file_path, skiprows=0)
rainfall= data.iloc[:,2].dropna().tolist() 
obs_rdii = data.iloc[:,1].tolist() 
delta_t = 600 #in sec ( for 10 min time step)
area_acres= 491.153

## Wighted flow with weighted score 

In [ ]:
''' First, Execute the corresponding Function (For ensmeble or Inividual ) '''

predicted_flows, average_flows, weighted_flow, metrics, final_weighted_score = RDII_all_algorithms_plot_with_weights(
    algorithm_data, 
    average_final, 
    delta_t, 
    rainfall, 
    area_acres, 
    obs_rdii, 
    weight_type='Rank_Weights',
   # plot_name="CCW_E6_2algo.svg",
)


## Function defination for ensemble configuration

In [39]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

def calculate_metrics(simulated, observed):
    """
    Calculate RMSE, R², PBIAS, and NSE between simulated and observed data.
    
    Parameters:
      simulated: Simulated flow time series.
      observed: Observed flow time series.
    
    Returns:
      metrics: Dictionary containing RMSE, R², PBIAS, and NSE.
    """
    simulated = np.array(simulated)
    observed = np.array(observed)
    
    rmse = np.sqrt(mean_squared_error(observed, simulated))
    r2 = r2_score(observed, simulated)
    pbias = 100 * np.sum(observed - simulated) / np.sum(observed)
    numerator = np.sum((observed - simulated) ** 2)
    denominator = np.sum((observed - np.mean(observed)) ** 2)
    nse = 1 - (numerator / denominator)
    
    return {
        'RMSE': rmse,
        'R2': r2,
        'PBIAS': pbias,
        'NSE': nse
    }

def transform_metrics(metrics):
    """
    Transform raw metrics into a 0-1 score for each metric.
    For RMSE and PBIAS (where lower is better) we use fixed functions,
    and for R² and NSE (where higher is better) we use a simple linear mapping.
    
    Parameters:
      metrics: dict with keys 'RMSE', 'R2', 'PBIAS', 'NSE'
      
    Returns:
      dict with transformed scores: f_RMSE, f_R2, f_PBIAS, f_NSE.
    """
    f_rmse = 1 / (1 + metrics['RMSE'])
    f_pbias = max(0, 1 - abs(metrics['PBIAS']) / 100)
    f_r2 = max(0, min(metrics['R2'], 1))
    nse = metrics['NSE']
    if nse < -1:
        f_nse = 0
    elif nse > 1:
        f_nse = 1
    else:
        f_nse = (nse + 1) / 2
        
    return {'f_RMSE': f_rmse, 'f_R2': f_r2, 'f_PBIAS': f_pbias, 'f_NSE': f_nse}

def composite_score(metrics):
    """
    Compute the composite score as the average of the four transformed metric scores.
    The final score is between 0 and 1, with higher values indicating better performance.
    """
    transformed = transform_metrics(metrics)
    score = (transformed['f_RMSE'] + transformed['f_R2'] + transformed['f_PBIAS'] + transformed['f_NSE']) / 4
    return score

def RDII_all_algorithms_plot_with_weights(algorithm_data, Rank_weight, delta_t, rainfall, Area, obs_rdii=None, weight_type='Linear_Weights', plot_name=None):
    """
    Perform RDII calculations, compute average flows, calculate weighted flows, and plot results.
    
    Additionally, if observed RDII data are provided, this function calculates:
      - Per-algorithm evaluation metrics (RMSE, R², PBIAS, NSE)
      - A composite final score for the weighted flow based on transformed metrics (each given 0.25 weight)
        The transformation functions map each metric to [0,1] such that higher is better.
    
    Parameters:
      algorithm_data: Dictionary containing parameter sets for all algorithms.
      Rank_weight: DataFrame containing weights for each algorithm.
      delta_t: Time step in seconds.
      rainfall: Rainfall time series (in inches).
      Area: Catchment area in acres.
      obs_rdii: Observed RDII time series (optional).
      weight_type: Type of weights to use ('Linear_Weights', 'Rank_Weights', or 'Softmax_Weights').
      plot_name: Optional string with filename (e.g., "my_plot.png") to save the plot.
    
    Returns:
      predicted_flows_all_algorithms: Dictionary where keys are algorithm names and values are lists of predicted flows.
      average_flows: Dictionary where keys are algorithm names and values are average flows.
      weighted_flow: The final weighted flow time series.
      metrics: Evaluation metrics (RMSE, R², PBIAS, NSE) for the weighted flow (if obs_rdii is provided; otherwise None).
      final_weighted_score: Composite score for the weighted flow (if obs_rdii is provided; otherwise None).
    """
    
    sns.set_style("whitegrid")
    
    predicted_flows_all_algorithms = {}
    average_flows = {}

    fig, ax1 = plt.subplots(figsize=(10, 6))

    time_values = [i * delta_t / 3600 for i in range(len(rainfall))]

    max_total_flow = 0
    max_flow_length = len(rainfall)

    algorithms = list(algorithm_data.keys())
    colors = plt.get_cmap('tab10').colors
    color_dict = {alg: colors[i % len(colors)] for i, alg in enumerate(algorithms)}

    for algorithm, all_params in algorithm_data.items():
        predicted_flows = []
        max_length = 0
        for params in all_params:
            (R1, T1, K1), (R2, T2, K2), (R3, T3, K3) = params

            uh1_ordinates = unit_hydrograph_ordinates(R1, T1, K1, delta_t)
            uh2_ordinates = unit_hydrograph_ordinates(R2, T2, K2, delta_t)
            uh3_ordinates = unit_hydrograph_ordinates(R3, T3, K3, delta_t)

            Q1_inch_sec = hydrograph_convolution(uh1_ordinates, rainfall)
            Q2_inch_sec = hydrograph_convolution(uh2_ordinates, rainfall)
            Q3_inch_sec = hydrograph_convolution(uh3_ordinates, rainfall)

            Q1_cfs = Q1_inch_sec * Area * 43560 / 12
            Q2_cfs = Q2_inch_sec * Area * 43560 / 12
            Q3_cfs = Q3_inch_sec * Area * 43560 / 12

            total_flow = add_flow(Q1_cfs, Q2_cfs, Q3_cfs)

            total_flow_m3s = np.array(total_flow) * 0.0283168  # Convert cfs to m3/s
            max_length = max(max_length, len(total_flow_m3s))
            predicted_flows.append(total_flow_m3s)

        padded_flows = [np.pad(flow, (0, max_length - len(flow)), 'constant') for flow in predicted_flows]
        max_flow_length = max(max_flow_length, max_length)
        average_flow = np.mean(padded_flows, axis=0)
        average_flows[algorithm] = average_flow
        predicted_flows_all_algorithms[algorithm] = padded_flows
        max_total_flow = max(max_total_flow, max(average_flow))

    time_values = [i * delta_t / 3600 for i in range(max_flow_length)]

    for algorithm, avg_flow in average_flows.items():
        average_flows[algorithm] = np.pad(avg_flow, (0, max_flow_length - len(avg_flow)), 'constant')

    weights = Rank_weight.set_index('Algorithm')[weight_type].to_dict()

    weighted_flow = np.zeros(max_flow_length)
    for algorithm, avg_flow in average_flows.items():
        weighted_flow += avg_flow * weights[algorithm]

    # for algorithm, flows in predicted_flows_all_algorithms.items():
    #     for flow in flows:
    #         ax1.plot(time_values[:len(flow)], flow, color=color_dict[algorithm], linestyle=':', alpha=0.5)

    for algorithm, avg_flow in average_flows.items():
        ax1.plot(time_values[:len(avg_flow)], avg_flow, label=f"Avg. Simulated RDII ({algorithm})", 
                 color=color_dict[algorithm], linestyle='-', linewidth=2)
# for Ensemble
    ax1.plot(time_values[:len(weighted_flow)], weighted_flow, label="Weighted Sim. RDII", 
             color='black', linewidth=2, linestyle='-', marker='D', alpha=0.9, markersize=5, markevery=10)

    metrics = None
    final_weighted_score = None

    if obs_rdii is not None:
        obs_rdii_m3s = np.array(obs_rdii) * 0.0283168  # Convert to m3/s
        n_sim = len(weighted_flow)
        n_obs = len(obs_rdii_m3s)
        n_final = max(n_sim, n_obs)
        if n_sim < n_final:
            weighted_flow = np.pad(weighted_flow, (0, n_final - n_sim), 'constant')
        if n_obs < n_final:
            obs_rdii_m3s = np.pad(obs_rdii_m3s, (0, n_final - n_obs), 'constant')
        time_obs = [i * delta_t / 3600 for i in range(n_final)]

        ax1.plot(time_obs, obs_rdii_m3s, label="Observed RDII", color='green', linewidth=2,
                 linestyle='--')

        metrics = calculate_metrics(weighted_flow, obs_rdii_m3s)
        final_weighted_score = composite_score(metrics)

        metrics_text = (
            f"Metrics\n"
            f"RMSE= {metrics['RMSE']:.4f}\n"
            f"R²= {metrics['R2']:.4f}\n"
            f"PBIAS= {metrics['PBIAS']:.4f}%\n"
            f"NSE= {metrics['NSE']:.4f}\n\n"
            f"Final Score= {final_weighted_score:.4f}"
        )
        ax1.text(0.60, 0.60, metrics_text, transform=ax1.transAxes, fontsize=10,
                 verticalalignment='top', horizontalalignment='left',
                 bbox=dict(boxstyle='round', facecolor='white', edgecolor='black', alpha=0.8))

    ax2 = ax1.twinx()
    rainfall_mm = np.array(rainfall) * 25.4  # Convert inches to mm
    rainfall_padded = np.pad(rainfall_mm, (0, max_flow_length - len(rainfall_mm)), 'constant')[:max_flow_length]
    time_rain = time_values[:max_flow_length]

    markerline, stemlines, baseline = ax2.stem(time_rain, rainfall_padded,
                                               linefmt='blue', markerfmt=' ', basefmt=' ', label='Rainfall')
    plt.setp(stemlines, 'color', 'blue', 'alpha', 0.5)
    plt.setp(markerline, 'color', 'blue', 'alpha', 0.5)
    ax2.set_ylim(25, 0)
    ax2.grid(False)

   # ax1.set_ylim(0, max_total_flow * 1.5)
    ax1.set_ylim(0, 0.10)
    ax1.set_xlim(0, 50)
    plt.xlim(left=0)

    ax1.set_xlabel('Time (Hours)', fontsize=12)
    ax1.set_ylabel('RDII (m³/s)', fontsize=12)
    ax2.set_ylabel('Rainfall (mm)', fontsize=12)

    ax1.legend(loc='upper right', bbox_to_anchor=(0.98, 0.98), borderaxespad=0.1, frameon=True, fontsize=10)
    ax2.legend(loc='upper right', bbox_to_anchor=(0.98, 0.90), borderaxespad=0.1, frameon=True, fontsize=10)

    ax1.grid(True)
    plt.tight_layout()

    if plot_name is not None:
        fig.savefig(plot_name)

    plt.show()

    return predicted_flows_all_algorithms, average_flows, weighted_flow, metrics, final_weighted_score

## Function defination (Individual algorithms)

In [80]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

def calculate_metrics(simulated, observed):
    """
    Calculate RMSE, R², PBIAS, and NSE between simulated and observed data.
    
    Parameters:
      simulated: Simulated flow time series.
      observed: Observed flow time series.
    
    Returns:
      metrics: Dictionary containing RMSE, R², PBIAS, and NSE.
    """
    simulated = np.array(simulated)
    observed = np.array(observed)
    
    rmse = np.sqrt(mean_squared_error(observed, simulated))
    r2 = r2_score(observed, simulated)
    pbias = 100 * np.sum(observed - simulated) / np.sum(observed)
    numerator = np.sum((observed - simulated) ** 2)
    denominator = np.sum((observed - np.mean(observed)) ** 2)
    nse = 1 - (numerator / denominator)
    
    return {
        'RMSE': rmse,
        'R2': r2,
        'PBIAS': pbias,
        'NSE': nse
    }

def transform_metrics(metrics):
    """
    Transform raw metrics into a 0-1 score for each metric.
    For RMSE and PBIAS (where lower is better) we use fixed functions,
    and for R² and NSE (where higher is better) we use a simple linear mapping.
    
    Parameters:
      metrics: dict with keys 'RMSE', 'R2', 'PBIAS', 'NSE'
      
    Returns:
      dict with transformed scores: f_RMSE, f_R2, f_PBIAS, f_NSE.
    """
    f_rmse = 1 / (1 + metrics['RMSE'])
    f_pbias = max(0, 1 - abs(metrics['PBIAS']) / 100)
    f_r2 = max(0, min(metrics['R2'], 1))
    nse = metrics['NSE']
    if nse < -1:
        f_nse = 0
    elif nse > 1:
        f_nse = 1
    else:
        f_nse = (nse + 1) / 2
        
    return {'f_RMSE': f_rmse, 'f_R2': f_r2, 'f_PBIAS': f_pbias, 'f_NSE': f_nse}

def composite_score(metrics):
    """
    Compute the composite score as the average of the four transformed metric scores.
    The final score is between 0 and 1, with higher values indicating better performance.
    """
    transformed = transform_metrics(metrics)
    score = (transformed['f_RMSE'] + transformed['f_R2'] + transformed['f_PBIAS'] + transformed['f_NSE']) / 4
    return score

def RDII_all_algorithms_plot_with_weights(algorithm_data, Rank_weight, delta_t, rainfall, Area, obs_rdii=None, weight_type='Linear_Weights', plot_name=None):
    """
    Perform RDII calculations, compute average flows, calculate weighted flows, and plot results.
    
    Additionally, if observed RDII data are provided, this function calculates:
      - Per-algorithm evaluation metrics (RMSE, R², PBIAS, NSE)
      - A composite final score for the weighted flow based on transformed metrics (each given 0.25 weight)
        The transformation functions map each metric to [0,1] such that higher is better.
    
    Parameters:
      algorithm_data: Dictionary containing parameter sets for all algorithms.
      Rank_weight: DataFrame containing weights for each algorithm.
      delta_t: Time step in seconds.
      rainfall: Rainfall time series (in inches).
      Area: Catchment area in acres.
      obs_rdii: Observed RDII time series (optional).
      weight_type: Type of weights to use ('Linear_Weights', 'Rank_Weights', or 'Softmax_Weights').
      plot_name: Optional string with filename (e.g., "my_plot.png") to save the plot.
    
    Returns:
      predicted_flows_all_algorithms: Dictionary where keys are algorithm names and values are lists of predicted flows.
      average_flows: Dictionary where keys are algorithm names and values are average flows.
      weighted_flow: The final weighted flow time series.
      metrics: Evaluation metrics (RMSE, R², PBIAS, NSE) for the weighted flow (if obs_rdii is provided; otherwise None).
      final_weighted_score: Composite score for the weighted flow (if obs_rdii is provided; otherwise None).
    """
    
    sns.set_style("whitegrid")
    
    predicted_flows_all_algorithms = {}
    average_flows = {}

    fig, ax1 = plt.subplots(figsize=(10, 6))

    time_values = [i * delta_t / 3600 for i in range(len(rainfall))]

    max_total_flow = 0
    max_flow_length = len(rainfall)

    algorithms = list(algorithm_data.keys())
    colors = plt.get_cmap('tab10').colors
    color_dict = {alg: colors[i % len(colors)] for i, alg in enumerate(algorithms)}

    for algorithm, all_params in algorithm_data.items():
        predicted_flows = []
        max_length = 0
        for params in all_params:
            (R1, T1, K1), (R2, T2, K2), (R3, T3, K3) = params

            uh1_ordinates = unit_hydrograph_ordinates(R1, T1, K1, delta_t)
            uh2_ordinates = unit_hydrograph_ordinates(R2, T2, K2, delta_t)
            uh3_ordinates = unit_hydrograph_ordinates(R3, T3, K3, delta_t)

            Q1_inch_sec = hydrograph_convolution(uh1_ordinates, rainfall)
            Q2_inch_sec = hydrograph_convolution(uh2_ordinates, rainfall)
            Q3_inch_sec = hydrograph_convolution(uh3_ordinates, rainfall)

            Q1_cfs = Q1_inch_sec * Area * 43560 / 12
            Q2_cfs = Q2_inch_sec * Area * 43560 / 12
            Q3_cfs = Q3_inch_sec * Area * 43560 / 12

            total_flow = add_flow(Q1_cfs, Q2_cfs, Q3_cfs)

            total_flow_m3s = np.array(total_flow) * 0.0283168  # Convert cfs to m3/s
            max_length = max(max_length, len(total_flow_m3s))
            predicted_flows.append(total_flow_m3s)

        padded_flows = [np.pad(flow, (0, max_length - len(flow)), 'constant') for flow in predicted_flows]
        max_flow_length = max(max_flow_length, max_length)
        average_flow = np.mean(padded_flows, axis=0)
        average_flows[algorithm] = average_flow
        predicted_flows_all_algorithms[algorithm] = padded_flows
        max_total_flow = max(max_total_flow, max(average_flow))

    time_values = [i * delta_t / 3600 for i in range(max_flow_length)]

    for algorithm, avg_flow in average_flows.items():
        average_flows[algorithm] = np.pad(avg_flow, (0, max_flow_length - len(avg_flow)), 'constant')

    weights = Rank_weight.set_index('Algorithm')[weight_type].to_dict()

    weighted_flow = np.zeros(max_flow_length)
    for algorithm, avg_flow in average_flows.items():
        weighted_flow += avg_flow * weights[algorithm]

    # for algorithm, flows in predicted_flows_all_algorithms.items():
    #     for flow in flows:
    #         ax1.plot(time_values[:len(flow)], flow, color=color_dict[algorithm], linestyle=':', alpha=0.5)

    for algorithm, avg_flow in average_flows.items():
        ax1.plot(time_values[:len(avg_flow)], avg_flow, label=f"Avg. Simulated RDII ({algorithm})", 
                 color=color_dict[algorithm], linestyle='-', linewidth=2)
# for Ensemble
    # ax1.plot(time_values[:len(weighted_flow)], weighted_flow, label="Weighted Sim. RDII", 
    #          color='black', linewidth=2, linestyle='-', marker='D', alpha=0.9, markersize=5, markevery=10)

    metrics = None
    final_weighted_score = None

    if obs_rdii is not None:
        obs_rdii_m3s = np.array(obs_rdii) * 0.0283168  # Convert to m3/s
        n_sim = len(weighted_flow)
        n_obs = len(obs_rdii_m3s)
        n_final = max(n_sim, n_obs)
        if n_sim < n_final:
            weighted_flow = np.pad(weighted_flow, (0, n_final - n_sim), 'constant')
        if n_obs < n_final:
            obs_rdii_m3s = np.pad(obs_rdii_m3s, (0, n_final - n_obs), 'constant')
        time_obs = [i * delta_t / 3600 for i in range(n_final)]

        ax1.plot(time_obs, obs_rdii_m3s, label="Observed RDII", color='green', linewidth=2,
                 linestyle='--')

        metrics = calculate_metrics(weighted_flow, obs_rdii_m3s)
        final_weighted_score = composite_score(metrics)

        metrics_text = (
            f"Metrics\n"
            f"RMSE= {metrics['RMSE']:.4f}\n"
            f"R²= {metrics['R2']:.4f}\n"
            f"PBIAS= {metrics['PBIAS']:.4f}%\n"
            f"NSE= {metrics['NSE']:.4f}\n\n"
            f"Final Score= {final_weighted_score:.4f}"
        )
        ax1.text(0.60, 0.60, metrics_text, transform=ax1.transAxes, fontsize=10,
                 verticalalignment='top', horizontalalignment='left',
                 bbox=dict(boxstyle='round', facecolor='white', edgecolor='black', alpha=0.8))

    ax2 = ax1.twinx()
    rainfall_mm = np.array(rainfall) * 25.4  # Convert inches to mm
    rainfall_padded = np.pad(rainfall_mm, (0, max_flow_length - len(rainfall_mm)), 'constant')[:max_flow_length]
    time_rain = time_values[:max_flow_length]

    markerline, stemlines, baseline = ax2.stem(time_rain, rainfall_padded,
                                               linefmt='blue', markerfmt=' ', basefmt=' ', label='Rainfall')
    plt.setp(stemlines, 'color', 'blue', 'alpha', 0.5)
    plt.setp(markerline, 'color', 'blue', 'alpha', 0.5)
    ax2.set_ylim(10, 0)
    ax2.grid(False)

    #ax1.set_ylim(0, max_total_flow * 1.5)
    ax1.set_ylim(0, 0.25)
    ax1.set_xlim(0, 50)
    plt.xlim(left=0)

    ax1.set_xlabel('Time (Hours)', fontsize=12)
    ax1.set_ylabel('RDII (m³/s)', fontsize=12)
    ax2.set_ylabel('Rainfall (mm)', fontsize=12)

    ax1.legend(loc='upper right', bbox_to_anchor=(0.98, 0.98), borderaxespad=0.1, frameon=True, fontsize=10)
    ax2.legend(loc='upper right', bbox_to_anchor=(0.98, 0.90), borderaxespad=0.1, frameon=True, fontsize=10)

    ax1.grid(True)
    plt.tight_layout()

    if plot_name is not None:
        fig.savefig(plot_name)

    plt.show()

    return predicted_flows_all_algorithms, average_flows, weighted_flow, metrics, final_weighted_score